In [61]:
# news
import pandas as pd
import os
from tqdm import tqdm
tqdm.pandas()
file_paths = []
for file in os.listdir(r"/home/qingzhi/wikipedia/official/new/content/GDELT"):
    if file.startswith("news"):
        file_paths.append(r"/home/qingzhi/wikipedia/official/new/content/GDELT/" + file)
file_paths

df_list = []
for x in tqdm(file_paths):
    df = pd.read_csv(x)
    # df = df.dropna()
    # df = df.groupby("company")['content'].agg(lambda x: ','.join(x)).reset_index()
    df_list.append(df)

dfs = pd.concat(df_list)

dfs



100%|███████████████████████████████████████████| 19/19 [00:15<00:00,  1.21it/s]


,Unnamed: 0,Unnamed: 0.1,company,date,news_url,lang,polarity,magnitude,score,numMentions,avgSalience,wiki_name,mid,wikipediaUrl,content
0,2497461,2497461,Intel Corp,2021-03-23T23:33:05Z,https://www.marketwatch.com/press-release/intel-ceo-pat-gelsinger-announces-idm-20-strategy-for-manufacturing-innovation-and-product-leadership-2021-03-23,en,0,19.1,0.3,62.0,0.019416,Intel,/m/03s7h,https://en.wikipedia.org/wiki/Intel,NaN
1,2497462,2497462,QUALCOMM Inc,2021-03-23T23:33:13Z,https://www.androidheadlines.com/2021/03/qualcomm-gaming-console-android-12-nintendo-switch-mobile-hardware.html,en,0,9.5,-0.1,14.0,0.643596,Qualcomm,/m/01m1xf,https://en.wikipedia.org/wiki/Qualcomm,Sign Up! envelope_alt Get the latest Android News in your inbox everyday arrow_right Android Apps & Games / Qualcomm Plans To Enter The Gaming Space With Its Own Console Qualcomm isn’t exactl...
2,2497476,2497476,American Electric Power Co Inc,2021-03-23T23:33:32Z,https://reliefweb.int/report/uganda/refugee-status-gender-and-learning,en,0,3.8,0.0,7.0,0.310116,AEP,/m/05khjc,https://en.wikipedia.org/wiki/American_Electric_Power,"\n\n\nRW COVID-19 page: Find latest updates on global humanitarian responses\n Find help on how to use the site, read terms and conditions, view the FAQs and API documentation. Access your account..."
3,2497478,2497478,Twitter Inc,2021-03-23T23:32:37Z,https://www.folhavitoria.com.br/entretenimento/noticia/03/2021/internautas-editam-informacoes-de-nascimento-de-tiago-leifert-e-mariana-ximenes-e-brincam-que-os-dois-sao-a-mesma-pessoa,pt,0,2.3,0.1,1.0,0.020012,Twitter,/m/0289n8t,https://de.wikipedia.org/wiki/Twitter,"\nEstadÃ£o ConteÃºdo\n\n\n\n O meme da semalhanÃ§a entre Tiago Leifert e Mariana Ximenes jÃ¡ existe faz tempo, mas na Ãºltima segunda-feira, dia 22, ele ganhou destaque nas redes sociais novamente..."
4,2497480,2497480,Twitter Inc,2021-03-23T23:32:17Z,https://www.tsf.pt/mundo/netanyahu-reclama-grande-vitoria-em-israel-13492338.html,pt,0,2.3,0.2,1.0,0.015197,Twitter,/m/0289n8t,https://de.wikipedia.org/wiki/Twitter,"O partido Likud, do atual primeiro-ministro israelita, foi o mais votado nas eleiÃ§Ãµes desta terÃ§a-feira. Â©Â EPA Benjamin Netanyahu reclama ""grande vitÃ³ria da direita"" nas eleiÃ§Ãµes legislat..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27741,3645056,1020887,Microsoft Corp,2021-11-07T22:31:30Z,http://www.voxy.co.nz/national/5/395132,en,0,13.3,0.2,3.0,0.051787,Microsoft,/m/04sv4,https://en.wikipedia.org/wiki/Microsoft,"New research from Microsoft ANZ shows many New Zealanders and Australians have been struggling to find motivation and joy in the things they love during the last 18 months, but would like to achie..."
27742,3645059,1020890,Tesla Inc,2021-11-07T22:31:34Z,https://www.ctvnews.ca/sci-tech/twitter-users-say-yes-to-musk-s-proposal-to-sell-10-per-cent-of-his-tesla-stock-1.5655947,en,0,8.6,-0.2,11.0,0.134620,Tesla,/m/0dr90d,"https://en.wikipedia.org/wiki/Tesla,_Inc.","FOLLOW ON Advertisement \nReuters\nStaff\n \nContact\n Elon Musk, Tesla CEO, arrives at an open house at the Tesla Gigafactory in Gruenheide, east of Berlin, Saturday Oct. 9, 2021, as the first ve..."
27743,3645060,1020891,Tesla Inc,2021-11-07T22:31:28Z,https://www.ndtv.com/world-news/twitter-users-say-yes-to-elon-musks-offer-to-sell-10-of-tesla-stock-2602505,en,0,8.8,-0.2,11.0,0.072949,Tesla,/m/0dr90d,"https://en.wikipedia.org/wiki/Tesla,_Inc.","""I was prepared to accept either outcome,"" Elon Musk said, after the voting ended. (File) Tesla Inc CEO Elon Musk should sell about 10% of his Tesla stock, according to 57.9% of people who voted o..."
27744,3645065,1020896,Facebook Inc,2021-11-07T22:31:34Z,https://www.news-daily.com/news/business/the-next-front-in-facebooks-misinformation-battle-climate-change/article_d394db17-e82a-5e61-845f-0393ffd97d09.html,en,0,25.0,-0.3,21.0,0.703209,Facebook,/m/02y1vz,https://en.wikipedia.org/wiki/Facebook,NaN


In [62]:
dfs["overall_score"] = dfs["score"] * dfs["magnitude"] * dfs["numMentions"]
dfs = dfs.sort_values("overall_score")

In [63]:
dfs2 = dfs[["company","date", "news_url","lang","magnitude","score","numMentions","avgSalience", "overall_score", "wikipediaUrl"]]
#.groupby("company").head(10)
# dfs2 = dfs2[dfs2.overall_score <= -30]
# dfs2

In [103]:
import numpy as np
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

p5 = dfs2[["company","overall_score"]].groupby("company").agg(percentile(1)).reset_index().rename(columns={"overall_score":"p5"})
p95 = dfs2[["company","overall_score"]].groupby("company").agg(percentile(99)).reset_index().rename(columns={"overall_score":"p95"})
p5

,company,p5
0,3D Systems Corp,-31.6701
1,3M Co,-174.4200
2,5N Plus Inc,0.2034
3,A O Smith Corp,-9.6795
4,ABB Ltd,-3.2192
...,...,...
1487,eHealth Inc,2.2800
1488,iHeartMedia Inc,-1.9800
1489,salesforce.com Inc,-23.2144
1490,thyssenkrupp AG,-8.3905


In [106]:
dfs3 = dfs2.merge(p5,on="company").merge(p95, on="company")


In [107]:
dfs4 = dfs3[(dfs3.overall_score <= dfs3.p5)|(dfs3.overall_score >= dfs3.p95)]

In [108]:
dfs4

,company,date,news_url,lang,magnitude,score,numMentions,avgSalience,overall_score,wikipediaUrl,p5,p95
0,Apple Inc,2021-05-23T20:17:28Z,https://appleinsider.com/articles/20/08/23/apple-versus-epic-games-fortnite-app-store-saga----the-story-so-far,en,314.2,-0.3,406.0,0.382625,-38269.56,https://en.wikipedia.org/wiki/Apple_Inc.,-144.1232,121.9860
1,Apple Inc,2021-04-26T16:19:06Z,https://appleinsider.com/articles/21/04/26/app-store-hey-and-antitrust-an-interview-with-david-heinemeier-hansson-on-the-appleinsider-podcast,en,295.1,-0.2,158.0,0.292771,-9325.16,https://en.wikipedia.org/wiki/Apple_Inc.,-144.1232,121.9860
2,Apple Inc,2021-06-07T02:31:51Z,https://www.msn.com/en-us/news/technology/apple-s-tightly-controlled-app-store-is-teeming-with-scams/ar-AAKL0TG,en,82.3,-0.3,77.0,0.734105,-1901.13,https://en.wikipedia.org/wiki/Apple_Inc.,-144.1232,121.9860
3,Apple Inc,2021-04-29T14:34:24Z,https://www.androidpolice.com/2012/07/27/a-complete-overview-of-the-apple-v-samsung-trial-things-arent-looking-great-for-androids-biggest-manufacturer/,en,68.4,-0.3,86.0,0.581341,-1764.72,https://en.wikipedia.org/wiki/Apple_Inc.,-144.1232,121.9860
4,Apple Inc,2021-06-18T12:48:35Z,https://www.nytimes.com/2021/06/14/podcasts/the-daily/apple-china-privacy.html,en,115.1,-0.1,138.0,0.556265,-1588.38,https://en.wikipedia.org/wiki/Apple_Inc.,-144.1232,121.9860
...,...,...,...,...,...,...,...,...,...,...,...,...
736934,GN Store Nord AS,2021-05-06T05:03:11Z,https://www.globenewswire.com/news-release/2021/05/06/2224031/0/en/Interim-Report-Q1-2021-GN-Store-Nord-46-organic-revenue-growth-GN-Audio-82-GN-Hearing-1-New-share-buyback-of-DKK-2-5-billion-init...,en,6.8,0.2,12.0,0.027969,16.32,https://en.wikipedia.org/wiki/GN_Store_Nord,16.3584,20.1216
736935,GN Store Nord AS,2021-10-29T06:17:29Z,https://www.finanznachrichten.de/nachrichten-2021-10/54337031-dgap-adhoc-gn-store-nord-a-s-interim-report-in-q3-2021-gn-store-nord-delivered-2-organic-revenue-growth-and-announced-the-acquisition-...,en,9.6,0.1,21.0,0.032246,20.16,https://en.wikipedia.org/wiki/GN_Store_Nord,16.3584,20.1216
736936,CSC Holdings LLC,2021-11-23T18:18:00Z,https://www.prnewswire.com/news-releases/pef-services-wins-client-service-award-at-2021-global-custodian-industry-leaders-event-301431049.html,en,8.7,0.3,7.0,0.185072,18.27,https://en.wikipedia.org/wiki/Corporation_Service_Company,18.2700,18.2700
736937,Reliance Worldwide Corp Ltd,2021-11-19T13:31:49Z,https://www.prnewswire.com:443/news-releases/rwc-acquires-ez-flo-international-makers-of-ez-flo-and-eastman-products-301429039.html,en,7.1,0.4,10.0,0.172147,28.40,https://en.wikipedia.org/wiki/Reliance_Worldwide_Corporation,28.4152,29.9048


In [102]:
# dfs4[dfs4.company == "CSC Holdings LLC"]

In [44]:
# dfs2[dfs2.overall_score <= dfs2.overall_score.quantile(0.05)]
# dfs2[dfs2.overall_score >= dfs2.overall_score.quantile(0.95)]

16.92

In [58]:

# import pandas as pd

# msci = pd.read_csv("./data/msci.csv")
# msci2 = pd.read_csv("./data/msci2.csv")

# # variable = 'GICS Industry'
# # variable2 = "SDG_01_NET_ALIGNMENT_SCORE"
# # variable3 = 'SDG_01_ALIGNMENT_SUMMARY'
# # variable4 = 'SDG_01_OPER_ALIGNMENT_SCORE'
# variable5 = "SDG_01_OPER_ALIGNMENT" # another thing

# SDG1 = msci[["Company Name", "Company ID"]].dropna()
# SDG2 = msci2[["ISSUER_NAME", "Figi", variable5]].dropna()

# df_label = SDG1.merge(SDG2, left_on="Company ID", right_on="Figi")[["Company Name", variable5]]
# df_label = df_label.rename(columns = {"Company Name": "company"})
# df_label

In [57]:
# df_label[df_label.SDG_01_OPER_ALIGNMENT=="Aligned"]

In [60]:
# dfs2[dfs2.company=="A O Smith Corp"]

,company,date,news_url,lang,magnitude,score,numMentions,avgSalience,overall_score,wikipediaUrl
4979,A O Smith Corp,2021-01-13T09:17:33Z,https://www.mouthshut.com/review/AO-Smith-HSE-HAS-X-15-L-Storage-Water-Geyser-review-nusmomtnpso,en,17.7,-0.3,2.0,0.014949,-10.62,https://en.wikipedia.org/wiki/A._O._Smith
29,A O Smith Corp,2021-01-01T00:01:50Z,https://www.theglobeandmail.com/investing/investment-ideas/article-bullish-on-ao-smith-corp/,en,1.7,-0.1,1.0,0.063778,-0.17,https://en.wikipedia.org/wiki/A._O._Smith
27792,A O Smith Corp,2021-04-29T11:49:06Z,https://www.chron.com/business/article/A-O-Smith-Q1-Earnings-Snapshot-16138136.php,en,3.2,0.0,3.0,0.019921,0.00,https://en.wikipedia.org/wiki/A._O._Smith
27702,A O Smith Corp,2021-04-29T11:47:29Z,https://www.sfgate.com/business/article/A-O-Smith-Q1-Earnings-Snapshot-16138136.php,en,3.2,0.0,3.0,0.019921,0.00,https://en.wikipedia.org/wiki/A._O._Smith
56256,A O Smith Corp,2021-10-28T14:32:38Z,https://www.investing.com/equities/a.o-smith-corp,en,0.7,0.0,2.0,0.248633,0.00,https://en.wikipedia.org/wiki/A._O._Smith
14631,A O Smith Corp,2021-04-15T11:49:18Z,https://www.investegate.co.uk/jupiter-green-invtst--jgc-/rns/portfolio-update/202104151223036218V/,en,0.1,0.0,1.0,0.011199,0.00,https://en.wikipedia.org/wiki/A._O._Smith
42502,A O Smith Corp,2021-01-28T18:17:40Z,https://www.investing.com/equities/a.o-smith-corp,en,0.6,0.0,1.0,0.091612,0.00,https://en.wikipedia.org/wiki/A._O._Smith
39220,A O Smith Corp,2021-01-28T12:34:50Z,https://www.chron.com/business/article/A-O-Smith-4Q-Earnings-Snapshot-15904611.php,en,3.6,0.0,3.0,0.010843,0.00,https://en.wikipedia.org/wiki/A._O._Smith
31877,A O Smith Corp,2021-10-26T02:32:14Z,https://biztimes.com/a-o-smith-acquires-canadian-water-heater-maker-in-nearly-200-million-deal/,en,3.0,0.3,3.0,0.050240,2.70,https://en.wikipedia.org/wiki/A._O._Smith
3546,A O Smith Corp,2021-11-18T20:03:21Z,https://www.prweb.com/releases/top_5_finalists_named_for_the_2021_johnny_unitas_golden_arm_award/prweb18337410.htm,en,8.5,0.3,6.0,0.380366,15.30,https://en.wikipedia.org/wiki/A._O._Smith


In [28]:
# pd.set_option('display.max_colwidth', 200)
# dfs2[dfs2.company == "Samsung Electronics Co Ltd"]

In [29]:
# https://towardsdatascience.com/web-scraping-news-articles-in-python-9dd605799558
import sys
sys.path.append("/home/qingzhi/.conda/envs/python3.7/lib/python3.7/site-packages")

In [109]:
import grequests
from tqdm import tqdm
class Test(object):
    def __init__(self, urls):
        self.urls = urls

    def exception(self, request, exception):
        print("Problem: {}: {}".format(request.url, exception))

    def async1(self):
        results = grequests.map((grequests.get(u, timeout=10, stream=False) for u in self.urls), exception_handler=self.exception, size=20)
        # print(results)
        return results
    
# test = Test(news_data.url.values.tolist()[0:100])
# results = test.async1()
# results

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.parse import urljoin
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

content_list = []
for i in tqdm(list(chunks(range(0, len(dfs4.news_url.values)), 20))):
    urls = dfs4.news_url.values[i]
    test = Test(urls)
    results = test.async1()
    for req, url in zip(results, urls):
    # for req, url in tqdm(zip(results, urls), total=len(urls), leave=False):
        if req == None:
            content_list.append(None)
        elif req.status_code == 200:
            # using the BeaitifulSoup module
            soup = BeautifulSoup(req.text, 'html.parser')
            title_list = []
            for title in soup.find_all('title'):
                title_list.append(title.get_text().split("|")[0])
            content_list.append(" ".join(title_list))
        else:
            content_list.append(None)
            
# news_data["pdf_list"] = content_list
# content_list

  0%|                                                   | 0/934 [00:00<?, ?it/s]

Problem: https://www.androidpolice.com/2012/07/27/a-complete-overview-of-the-apple-v-samsung-trial-things-arent-looking-great-for-androids-biggest-manufacturer/: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Problem: https://www.washingtonpost.com/technology/2021/06/06/apple-app-store-scams-fraud/: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=10)
Problem: https://www.washingtonpost.com/technology/2021/04/30/epic-apple-trial-antitrust-fortnite-app-store/: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=10)


  0%|                                         | 2/934 [00:19<2:17:08,  8.83s/it]

Problem: https://www.hpr2.org/post/epic-games-vs-apple-trial-has-begun-heres-what-you-need-know: HTTPSConnectionPool(host='www.hpr2.org', port=443): Max retries exceeded with url: /post/epic-games-vs-apple-trial-has-begun-heres-what-you-need-know (Caused by SSLError(SSLCertVerificationError("hostname 'www.hpr2.org' doesn't match either of 'mail.hawaiipublicradio.org', 'www.mail.hawaiipublicradio.org', 'secure.hawaiipublicradio.org', 'members.hawaiipublicradio.org', 'www.hawaiipublicradio.org', 'files.hawaiipublicradio.org'")))


  0%|▏                                        | 3/934 [00:23<1:38:54,  6.37s/it]

...

 99%|██████████████████████████████████████▋| 926/934 [1:21:44<00:45,  5.69s/it]

Problem: https://www.jcnnewswire.com/pressrelease/70581/3/MC-and-Hokuden-to-form-Hydro-Power-Alliance-in-Hokkaidos-Donan-Region: HTTPSConnectionPool(host='www.jcnnewswire.com', port=443): Max retries exceeded with url: /pressrelease/70581/3/MC-and-Hokuden-to-form-Hydro-Power-Alliance-in-Hokkaidos-Donan-Region (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f53653e22b0>, 'Connection to www.jcnnewswire.com timed out. (connect timeout=10)'))
Problem: http://news.cnwest.com/tianxia/a/2021/04/21/19634162.html: HTTPConnectionPool(host='news.cnwest.com', port=80): Read timed out.


 99%|██████████████████████████████████████▊| 929/934 [1:22:18<00:42,  8.44s/it]

Problem: http://www.newratings.de/du/main/company_headline.m?nid=13202798: HTTPSConnectionPool(host='www.newratings.de', port=443): Max retries exceeded with url: /du/main/company_headline.m?nid=13202798 (Caused by SSLError(SSLCertVerificationError("hostname 'www.newratings.de' doesn't match either of '*.ariva-services.de', 'ariva-services.de'")))
Problem: https://www.bndes.gov.br/wps/portal/site/home/imprensa/noticias/conteudo/bndes-aprova-697-milhoes-para-duratex-adquirir-maquinas-industriais-e-equipamentos/!ut/p/z0/rY47TwMxEIR_C4XLla1w5Eh5IpFQHqIBKbiJNmeT2xCvfX4E-PeYEzUV3c5oZr-RWu6lZrzSCTN5xkvVr3p-2C42q8fmSW3V3XOjuuVt27y0K7W5n8m11H8H6gc6j6PupO49Z_uZ5f7IxqYDccqUSz-RhBq8s0KRC9FyQqHYZ-oJk1BTrxgv1FQEDNFfEeaLFhxdBl-tgBHBlIgVAGjGQpEiOKwHYwJiU1KOhJTAgq1uQGc5-_SzbxZ3D7uT1AHzUKNv_nfh_4LCuz5-fXQ33zM4as4!/: HTTPSConnectionPool(host='www.bndes.gov.br', port=443): Read timed out. (read timeout=10)


100%|███████████████████████████████████████| 934/934 [1:22:44<00:00,  5.32s/it]


In [111]:
dfs4["header"] = content_list

/tmp/ipykernel_6149/3521994212.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs4["header"] = content_list


In [114]:
dfs4

,company,date,news_url,lang,magnitude,score,numMentions,avgSalience,overall_score,wikipediaUrl,p5,p95,header
0,Apple Inc,2021-05-23T20:17:28Z,https://appleinsider.com/articles/20/08/23/apple-versus-epic-games-fortnite-app-store-saga----the-story-so-far,en,314.2,-0.3,406.0,0.382625,-38269.56,https://en.wikipedia.org/wiki/Apple_Inc.,-144.1232,121.9860,"Epic Games vs Apple trial, verdict, and aftermath - all you need to know"
1,Apple Inc,2021-04-26T16:19:06Z,https://appleinsider.com/articles/21/04/26/app-store-hey-and-antitrust-an-interview-with-david-heinemeier-hansson-on-the-appleinsider-podcast,en,295.1,-0.2,158.0,0.292771,-9325.16,https://en.wikipedia.org/wiki/Apple_Inc.,-144.1232,121.9860,"App Store, Hey, and antitrust: An interview with David Heinemeier Hansson on the AppleInsider Podcast"
2,Apple Inc,2021-06-07T02:31:51Z,https://www.msn.com/en-us/news/technology/apple-s-tightly-controlled-app-store-is-teeming-with-scams/ar-AAKL0TG,en,82.3,-0.3,77.0,0.734105,-1901.13,https://en.wikipedia.org/wiki/Apple_Inc.,-144.1232,121.9860,Apple’s tightly controlled App Store is teeming with scams
3,Apple Inc,2021-04-29T14:34:24Z,https://www.androidpolice.com/2012/07/27/a-complete-overview-of-the-apple-v-samsung-trial-things-arent-looking-great-for-androids-biggest-manufacturer/,en,68.4,-0.3,86.0,0.581341,-1764.72,https://en.wikipedia.org/wiki/Apple_Inc.,-144.1232,121.9860,None
4,Apple Inc,2021-06-18T12:48:35Z,https://www.nytimes.com/2021/06/14/podcasts/the-daily/apple-china-privacy.html,en,115.1,-0.1,138.0,0.556265,-1588.38,https://en.wikipedia.org/wiki/Apple_Inc.,-144.1232,121.9860,Apple’s Bet on China - The New York Times bars
...,...,...,...,...,...,...,...,...,...,...,...,...,...
736934,GN Store Nord AS,2021-05-06T05:03:11Z,https://www.globenewswire.com/news-release/2021/05/06/2224031/0/en/Interim-Report-Q1-2021-GN-Store-Nord-46-organic-revenue-growth-GN-Audio-82-GN-Hearing-1-New-share-buyback-of-DKK-2-5-billion-init...,en,6.8,0.2,12.0,0.027969,16.32,https://en.wikipedia.org/wiki/GN_Store_Nord,16.3584,20.1216,Interim Report Q1 2021: GN Store Nord +46% organic revenue
736935,GN Store Nord AS,2021-10-29T06:17:29Z,https://www.finanznachrichten.de/nachrichten-2021-10/54337031-dgap-adhoc-gn-store-nord-a-s-interim-report-in-q3-2021-gn-store-nord-delivered-2-organic-revenue-growth-and-announced-the-acquisition-...,en,9.6,0.1,21.0,0.032246,20.16,https://en.wikipedia.org/wiki/GN_Store_Nord,16.3584,20.1216,"DGAP-Adhoc: GN Store Nord A/S: Interim report: In Q3 2021, GN Store Nord delivered 2% organic revenue growth and announced the acquisition of SteelSeries. Revision of GN Audio's organic revenue gr..."
736936,CSC Holdings LLC,2021-11-23T18:18:00Z,https://www.prnewswire.com/news-releases/pef-services-wins-client-service-award-at-2021-global-custodian-industry-leaders-event-301431049.html,en,8.7,0.3,7.0,0.185072,18.27,https://en.wikipedia.org/wiki/Corporation_Service_Company,18.2700,18.2700,PEF Services Wins Client Service Award at 2021 Global Custodian Industry Leaders Event
736937,Reliance Worldwide Corp Ltd,2021-11-19T13:31:49Z,https://www.prnewswire.com:443/news-releases/rwc-acquires-ez-flo-international-makers-of-ez-flo-and-eastman-products-301429039.html,en,7.1,0.4,10.0,0.172147,28.40,https://en.wikipedia.org/wiki/Reliance_Worldwide_Corporation,28.4152,29.9048,"RWC Acquires EZ-FLO International, Makers of EZ-FLO™ and Eastman™ Products"


In [132]:
dfs4.to_csv("./temp_data2/news_headlines.csv")

In [4]:
import pandas as pd
dfs4 = pd.read_csv("news_headlines.csv")

In [5]:
def get_content_url(url):
    # url = "https://www.androidpolice.com/2012/07/27/a-complete-overview-of-the-apple-v-samsung-trial-things-arent-looking-great-for-androids-biggest-manufacturer/"
    return " ".join(url.split("/")[-2:]).replace("-"," ")

dfs4["header2"] = dfs4["news_url"].apply(get_content_url)

In [ ]:
dfs4.to_csv("./temp_data2/news_headlines.csv")

In [148]:
import pandas as pd
df2 = pd.read_csv("./temp_data2/news_headlines.csv").sort_values("overall_score")
len(df2.company.unique())

1492

In [149]:
import pandas as pd

# solution given by https://stackoverflow.com/questions/36880490/why-use-pandas-qcut-return-valueerror-bin-edges-must-be-unique
def pct_rank_qcut(series, n):
    edges = pd.Series([float(i) / n for i in range(n + 1)])
    f = lambda x: (edges >= x).values.argmax()
    return series.rank(pct=1).apply(f)



In [150]:
df_temp = df2["company"].value_counts().reset_index()
companies_to_include = df_temp[df_temp.company >= 10]["index"].values

In [151]:
df3 = df2[df2.company.isin(companies_to_include)]
df3['score_quantile'] = df3.groupby("company")["overall_score"].transform(lambda x: pct_rank_qcut(x,5))

C:\Users\qhuca\anaconda3\envs\python3.7\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [86]:
# df2['score_quantile'] = pct_rank_qcut(df2["overall_score"], 5)

In [152]:
df3["score_quantile"].unique()

array([1., 2., 3., 5., 4.])

In [153]:
df3 = df3[(df3.score_quantile==1)|(df3.score_quantile==5)]

In [154]:
len(df3.company.unique())

281

In [155]:
len(df2[~df2.company.isin(companies_to_include)].company.unique())

1209

In [179]:
df4 = df2[~df2.company.isin(companies_to_include)]
dfs4 = pd.concat([df3,df4])
dfs4 = dfs4.drop(columns=['score_quantile']).dropna()

In [95]:
# df_temp = df2["company"].value_counts().reset_index()
# companies_to_include = df_temp[df_temp.company >= 50]["index"].values
# df3 = df2[df2.company.isin(companies_to_include)]
# df3.groupby("company").apply(lambda x: pd.qcut(x["overall_score"], q=5, labels=range(3), duplicates='drop'))

In [181]:
import tqdm
import warnings
import difflib
import tqdm

warnings.filterwarnings('ignore')
dfs4_list = []
for company in tqdm.tqdm(dfs4.company.unique()):
    df_temp = dfs4[dfs4.company==company]
    # Threshold filter based on Percentage similarity
    thr = 0.55
    df_temp['Flag'] = 0
    for text in df_temp['header'].tolist():
        df_temp['temp'] = [difflib.SequenceMatcher(None, text1,text).ratio() for text1 in df_temp['header'].tolist()]
        df_temp.loc[df_temp['temp'].gt(thr),['Flag']] = df_temp['Flag'].max()+1
    df_temp.drop('temp',1)

    dfs4_list.append(df_temp.loc[~df_temp['Flag'].duplicated(keep='first')])

100%|██████████████████████████████████████████████████████████████████████████████| 1488/1488 [03:12<00:00,  7.73it/s]


In [183]:
pd.concat(dfs4_list).to_csv("./temp_data/news_headlines2.csv")

In [184]:
import pandas as pd
dfs4 = pd.read_csv("./temp_data/news_headlines2.csv")

In [186]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

df1 = dfs4[["company","magnitude", "score", "numMentions", "avgSalience", "overall_score"]].groupby("company").agg([np.sum, np.mean, np.std, np.median,
                     np.var, np.min, np.max, percentile(5), percentile(95), percentile(10), percentile(90)]).reset_index()
df1.columns = ['_'.join(col).strip() for col in df1.columns.values]
df1 = df1.rename(columns={"company_":"company"})

In [188]:
df1.to_csv("./temp_data/news_sentiment.csv")